# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# download nltk libraries
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# import libraries
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
import re

from sklearn.model_selection import train_test_split
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.metrics import classification_report,accuracy_score,f1_score,precision_score,recall_score
from sklearn.model_selection import GridSearchCV
import pickle

In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse1.db')
df = pd.read_sql("SELECT * FROM Disaster", engine)  
X = df.message.values
Y = df.drop(columns=['id', 'message', 'original', 'genre']).values
category_names = np.array(df.drop(columns=['id', 'message', 'original', 'genre']).columns)

### 2. Write a tokenization function to process your text data

In [4]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
def replace_urls(text):
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')
    return text

In [5]:
def tokenize(text):
    text=replace_urls(text)
    text=re.sub(r'[^a-zA-Z0-9]', ' ', text).lower()
    tokens = word_tokenize(text)
    words = [w for w in tokens if w not in stopwords.words("english")]
    lemmed = [WordNetLemmatizer().lemmatize(w, pos='v') for w in words]
    stem_words = [PorterStemmer().stem(w) for w in lemmed]
    return stem_words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
y_pred = pipeline.predict(X_test)

In [11]:
for ids in range(y_test.shape[-1]):
    print(classification_report(y_test[:,ids], y_pred[:,ids]))
    print("------------------------------------------------------\n")

             precision    recall  f1-score   support

          0       0.62      0.45      0.52      1494
          1       0.84      0.92      0.88      5015
          2       0.62      0.18      0.28        45

avg / total       0.79      0.81      0.79      6554

------------------------------------------------------

             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5453
          1       0.78      0.42      0.55      1101

avg / total       0.87      0.88      0.87      6554

------------------------------------------------------

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6526
          1       0.00      0.00      0.00        28

avg / total       0.99      1.00      0.99      6554

------------------------------------------------------

             precision    recall  f1-score   support

          0       0.75      0.86      0.80      3824
          1       0.75     

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
def get_scores(y_true, y_pred):
    """
    Returns the accuracy, precision and recall and f1 scores of the two same shape numpy arrays `y_true` and `y_pred`.

    INPUTS:
        y_true - Numpy array object - A (1 x n) vector of true values
        y_pred - Numpy array object - A (1 x n) vector of predicted values
        
    OUPUT:
        dict_scores - Python dict - A dictionary of accuracy, precision and recall and f1 scores of `y_true` and `y_pred`.
    """
    
    # Compute the accuracy score of y_true and y_pred
    accuracy = accuracy_score(y_true, y_pred)
    
    # Compute the precision score of y_true and y_pred
    precision =round( precision_score(y_true, y_pred, average='micro'))
    
    # Compute the recall score of y_true and y_pred
    recall = recall_score(y_true, y_pred, average='micro')
    
    # Compute the recall score of y_true and y_pred
    f_1 = f1_score(y_true, y_pred, average='micro')
    
    # A dictionary of accuracy, precision and recall and f1 scores of `y_true` and `y_pred`
    dict_scores = {
        'Accuracy': accuracy, 
        'Precision': precision, 
        'Recall': recall, 
        'F1 Score': f_1
    }
    
    return dict_scores

In [13]:
tabulate_metric_scores = lambda y_test, y_pred : pd.DataFrame([get_scores(y_test[:, ids], y_pred[:, ids]) for ids in range(y_test.shape[-1])], index=category_names)

tabulate_metric_scores(y_test, y_pred)


,Accuracy,F1 Score,Precision,Recall
related,0.805920,0.805920,1.0,0.805920
request,0.882972,0.882972,1.0,0.882972
offer,0.995728,0.995728,1.0,0.995728
aid_related,0.747788,0.747788,1.0,0.747788
medical_help,0.918828,0.918828,1.0,0.918828
medical_products,0.950107,0.950107,1.0,0.950107
search_and_rescue,0.972841,0.972841,1.0,0.972841
security,0.982453,0.982453,1.0,0.982453
military,0.965060,0.965060,1.0,0.965060
child_alone,1.000000,1.000000,1.0,1.000000


### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7ffa4edab488>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
               oob_score=False, random_state=None

In [16]:
parameters = {'tfidf__norm': ['l1','l2'],
              'clf__estimator__criterion': ["gini", "entropy"]
    
             }

cv = GridSearchCV(pipeline, param_grid=parameters)

In [ ]:
cv.fit(X_train,y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
y_pred = cv.predict(X_test)

In [16]:
tabulate_metric_scores = lambda y_test, y_pred : pd.DataFrame([get_scores(y_test[:, ids], y_pred[:, ids]) for ids in range(y_test.shape[-1])], index=category_names)

tabulate_metric_scores(y_test, y_pred)


,Accuracy,F1 Score,Precision,Recall
related,0.807751,0.807751,1.0,0.807751
request,0.881141,0.881141,1.0,0.881141
offer,0.994812,0.994812,1.0,0.994812
aid_related,0.747177,0.747177,1.0,0.747177
medical_help,0.924779,0.924779,1.0,0.924779
medical_products,0.947513,0.947513,1.0,0.947513
search_and_rescue,0.974214,0.974214,1.0,0.974214
security,0.983827,0.983827,1.0,0.983827
military,0.968874,0.968874,1.0,0.968874
child_alone,1.000000,1.000000,1.0,1.000000


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [19]:
with open('MLpipeline.pkl', 'wb') as file:
    pickle.dump(cv, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.